## Srovnání metod řešení least-squares problémů (LSQR, BCGS, BICG, CGNE)

In [5]:
import time
import pandas as pd
from petsc4py import PETSc

def solve_benchmark(matrix_path, method):
    # Funkce pro řešení lineárního systému pomocí zadané metody
    try:
        # Načtení dat
        viewer = PETSc.Viewer().createBinary(matrix_path, 'r')
        A = PETSc.Mat().load(viewer)
        
        # Příprava vektorů
        x_exact, b = A.createVecs()
        x_exact.set(1.0)
        A.mult(x_exact, b)
        x = x_exact.copy()
        x.set(0.0)
        
        # Nastavení solveru
        ksp = PETSc.KSP().create()
        ksp.setOperators(A)
        
        # Zde zkusíme nastavit metodu
        ksp.setType(method)
        ksp.getPC().setType('none')
        ksp.setTolerances(rtol=1e-6)
        
        # Výpočet
        start = time.perf_counter()
        ksp.solve(b, x)
        end = time.perf_counter()
        
        its = ksp.getIterationNumber()
        return {"Metoda": method, "Status": "OK", "Iterace": its, "Čas [s]": f"{end-start:.4f}"}
    
    except Exception as e:
        # Zachycení všech PETSc chyb
        error_msg = str(e).splitlines()[0] # Vezmeme jen první řádek chyby
        return {"Metoda": method, "Status": "Nepodporováno", "Iterace": "-", "Čas [s]": "-"}

Srování metod pro zvolené matice

In [6]:
matrices = ["data/ash219.dat", "data/well1850.dat", "data/illc1850.dat"]
methods = ["lsqr", "cgne", "bicg", "bcgs"]

results = []
for m_path in matrices:
    m_name = m_path.split('/')[-1]
    
    for method in methods:
        res = solve_benchmark(m_path, method)
        res["Matice"] = m_name
        results.append(res)

table = pd.DataFrame(results)
print("\nSROVNÁNÍ METOD")
display(table[["Matice", "Metoda", "Status", "Iterace", "Čas [s]"]])


SROVNÁNÍ METOD


,Matice,Metoda,Status,Iterace,Čas [s]
0,ash219.dat,lsqr,OK,18,0.0001
1,ash219.dat,cgne,Nepodporováno,-,-
2,ash219.dat,bicg,Nepodporováno,-,-
3,ash219.dat,bcgs,Nepodporováno,-,-
4,well1850.dat,lsqr,OK,358,0.0059
5,well1850.dat,cgne,Nepodporováno,-,-
6,well1850.dat,bicg,Nepodporováno,-,-
7,well1850.dat,bcgs,Nepodporováno,-,-
8,illc1850.dat,lsqr,OK,618,0.0086
9,illc1850.dat,cgne,Nepodporováno,-,-
